Set up field for the vector space to work in exact precision

In [192]:
P.<x1, x2, x3, x4, x5, x6> = QQ[]
P

Multivariate Polynomial Ring in x1, x2, x3, x4, x5, x6 over Rational Field

In [193]:
cos(pi/8).minpoly('x5')

x5^4 - x5^2 + 1/8

In [194]:
R.<t1, t2, t3, t4, t5, t6> = QuotientRing(P, P.ideal(cos(pi/4).minpoly('x1'), cos(pi/5).minpoly('x2'), cos(pi/6).minpoly('x3'), cos(pi/7).minpoly('x4'), cos(pi/8).minpoly('x5'), cos(pi/9).minpoly('x6')))
R

Quotient of Multivariate Polynomial Ring in x1, x2, x3, x4, x5, x6 over Rational Field by the ideal (x1^2 - 1/2, x2^2 - 1/2*x2 - 1/4, x3^2 - 3/4, x4^3 - 1/2*x4^2 - 1/2*x4 + 1/8, x5^4 - x5^2 + 1/8, x6^3 - 3/4*x6 - 1/8)

In [196]:
# Input the complete matrix, not just lower triangular

# Coxeter matrix are the values -cos(pi/m) (not just the usual m) but as elements of the polynomial ring

# Matrix for B2 Tilde
#coxeter_matrix = matrix([[1,(-1)*t1,0], [(-1)*t1,1,(-1)*t1],[0,(-1)*t1,1]])


# Matrix for B_3 Tilde
#coxeter_matrix = matrix([[1,(-1/2)*t,0,0], [(-1/2)*t,1,(-1/2),(-1/2)],[0,(-1/2),1,0],[0,(-1/2),0,1]])

# Matrix for G2
#coxeter_matrix = matrix([[1,(-1/2),0], [(-1/2),1,(-1)*t3],[0,(-1)*t3,1]])

# Matrix for 3,9
#coxeter_matrix = matrix([[1,(-1/2),0], [(-1/2),1,(-1)*t6],[0,(-1)*t6,1]])

# Matrix for 4,4,3,5
#coxeter_matrix = matrix([[1,(-1)*t1,0,0], [(-1)*t1,1,(-1/2),(-1)*t1],[0,(-1/2),1,(-1)*t2],[0,(-1)*t1, (-1)*t2, 1]])


# Matrix for C_3 tilde
coxeter_matrix = matrix([[1,(-1)*t1,0,0], [(-1)*t1,1,(-1/2),0],[0,(-1/2),1,(-1)*t1],[0,0,(-1)*t1, 1]])


In [197]:
coxeter_matrix

[   1  -t1    0    0]
[ -t1    1 -1/2    0]
[   0 -1/2    1  -t1]
[   0    0  -t1    1]

In [198]:
v = FreeModule(R,4)
v
dim_v = 4

In [199]:
def bilinear(x,y):
    '''Computes the bilinear form of two roots'''
    temp = 0
    for i in range(dim_v):
        for j in range(dim_v):
            temp += x[i]*y[j]*(coxeter_matrix[i,j])
    return temp

def s(i):
    '''Returns the ith simple root'''
    return v.basis()[i]

def simple_reflection(s,x):
    return x-2*bilinear(x,s)*s

def w_action(word_list,x):
    '''Input a word w in the generators as a list and a root \alpha. Returns w(\alpha)'''
    temp_x = x
    word_list.reverse()
    
    for s in word_list:
        temp_x = simple_reflection(s,temp_x)
    
    # reverse back as it reverses the word outside this function as well
    word_list.reverse()
    
    return temp_x

def inversion_set(w):
    '''Returns the left inversion set of an element w as a list'''
    inversionlist = [w[0]]

    for i in range(len(w)-1):        
        # note: w[0:i+1] does not include the endpoint i+1
        temp = w[0:i+1]
        inversionlist.append(w_action(temp,w[i+1]))
        
    return inversionlist

# Auxiliary function for computing elementary roots
def polynomial_evaluator(x):
    '''Given an element of the quotient ring, subs in value for indeterminate 
       (by lifting the element to the symbolic ring) and evaluates the function
       returning a float.'''
    
    if 't' in str(x):
        return RR(SR(str(x))(t1=cos(pi/4), t2=cos(pi/5), t3=cos(pi/6), t4=cos(pi/7), t5=cos(pi/8), t6=cos(pi/9) ))
    else:
        return RR(x)

def elementary_roots():
    '''Returns the set of elementary roots'''
    elementary = []
    
    for i in range(dim_v):
        elementary.append(s(i))
    
    temp_n = 1
    counter = 0
    
    while temp_n < len(elementary):
        
        new_root_counter = 0
        
        temp_n = len(elementary)

        for root in elementary[counter:]:
            for i in range(dim_v):

                temp = w_action([s(i)],root)
                
                temp_i = polynomial_evaluator(temp[i])
                root_i = polynomial_evaluator(root[i])

                if (temp_i > root_i) and (temp_i-root_i < 2) and temp not in elementary:
                    elementary.append(temp)
                    new_root_counter += 1
        
        counter = len(elementary) - new_root_counter - 1
                      
    return elementary


def elementary_inversion_set(w, elementary):
    '''returns the left elementary inversion set of an element, but compute the elementary roots first and input it, 
       to avoid recalculating it everytime.'''
    
    inver = inversion_set(w)
    
    e_inversion = [root for root in elementary if root in inver]
    
    return e_inversion

def left_length_increase(word, s):
    '''Tells you whether a word input as a list of simple roots, increases in length by multiplication by s on the left'''
    
    if word[0] == s:
        return False
    
    word_copy = word[:]
    word_copy.reverse()
    
    #temp_x = s
    
    #for t in word_copy:
    #   temp_x = simple_reflection(t,temp_x)
        
        # If the action of a subword on alpha_s is already negative, then the word is not reduced
    #   if sum(temp_x) < 0:
    #       return False
    
    root = w_action(word_copy, s)
    
    sum_root = 0
    
    for i in root:
        sum_root += polynomial_evaluator(i)
    
    if sum_root >= 0:
        return True
    else:
        return False

def words_are_same_element(word1, word2):
    '''Given two reduced words, determine whether they are the same element 
    by checking if their inversion sets are the same'''
    
    # this only works because we assume words are reduced
    if len(word1) != len(word2):
        return False
  
    w1_inversion = inversion_set(word1)
    w2_inversion = inversion_set(word2)
        
    for root in w1_inversion:
        if root in w2_inversion:
            w2_inversion.remove(root)
    
    if len(w2_inversion) == 0:
        return True
    else:
        return False

def word_nice(word):
    '''prints word (as a vector of simple roots) as a nice string of letters'''
    word_string = ''
    for vector in word:
        if vector == s(0):
            word_string += 's'
        elif vector == s(1):
            word_string += 't'
        elif vector == s(2):
            word_string += 'u'
        else:
            word_string += 'v'
    
    return word_string    

def wordnice_to_vectorlist(word):
    '''Given a nice presented word returns a list of simple vectors representing the word for computation'''
    word_list = []
    
    for letter in word:
        if letter == 's':
            word_list.append(s(0))
        if letter == 't':
            word_list.append(s(1))
        if letter == 'u':
            word_list.append(s(2))
        if letter == 'v':
            word_list.append(s(3))
    
    return word_list


def elementary_inversion_lists_same(e1, e2):
    '''given two elementary inversion lists, check whether they are the same as sets'''
    if len(e1) != len(e2):
        return False
    
    e1_copy = e1[:]
    
    for root in e1:
        if root in e2:
            e1_copy.remove(root)
        else:
            return False
    
    if len(e1_copy) == 0:
        return True
    else:
        return False

## BH-Automaton

In [200]:
def bh_automata():
    '''Returns the list of states, where each item is a list with first index a state
    and then following items in the list are tuples with a transition letter and target state'''

    # list of states and their transition information
    # as elementary inversion lists
    states_transitions = []
    
    # list of states only (as representative word)
    state_list = ['s', 't', 'u', 'v']
    
    counter = 0
    num_of_new_states = 1
    
    while num_of_new_states > 0:
        
        num_of_new_states = 0
        
        for state in state_list[counter:]:
            
            counter = len(state_list)-num_of_new_states
            
            state_vector = wordnice_to_vectorlist(state)
            
            state_elementary = elementary_inversion_set(state_vector, elementary)

            state_info = [state]
            
            for j in range(0,dim(v)):
                #print(' ')
                
                generator_word = word_nice([s(j)])
                #print('Word representing inversion set is: ' + state + ' generator is: ' + generator_word )
                
                if left_length_increase(state_vector,s(j)):

                    #print('Yes, ' + state + ' increases on the left by ' + word_nice([s(j)]))
                    
                    target_state = [s(j)] + state_vector
                    target_state_word = word_nice(target_state)
                    
                    #print('Word repping Target State: ' + target_state_word)
                    
                    target_state_elementary_inversion_set = elementary_inversion_set(target_state, elementary)
                    
                    # If target state elementary inversion set is not in the list of states, add to the list.
                    # if already in list, add appropriate transition
                    target_state_is_new = True
                    
                    for st in state_list:
                        st_elementary = elementary_inversion_set(wordnice_to_vectorlist(st), elementary)
                        
                        if elementary_inversion_lists_same(target_state_elementary_inversion_set, st_elementary):
                            #print('Its inversion set is already represented in state_list. It has same e-inversion set as ' + st)
                            
                            transition = (generator_word, st)
                            state_info.append(transition)
                            
                            #print('Added the transition pair (' + generator_word + ', ' + st )
                            
                            target_state_is_new = False
                            break
                    
                    if target_state_is_new == True:
                        # add target_state rep to state list
                        state_list.append(target_state_word)
                        #print('New State. Adding to state_list, the rep: ' + word_nice(target_state))
                        num_of_new_states += 1
                                
                        transition = (generator_word, target_state_word)
                        state_info.append(transition)
                        #print('Added the transition pair (' + generator_word + ', ' + target_state_word )

                #else:
                    #print('No, ' + state + ' decreases by ' + word_nice([s(j)]))
                    
            states_transitions.append(state_info)
        
         
        #print('num_of_new_states: ' + str(num_of_new_states))
        print('NUMBER OF STATES IS NOW: ' + str(len(state_list)))
        #print('State reps are: ')
        #print(state_list)
        #print('The COUNTER is at: ' + str(counter))
        
    print('Number of states: ' + str(len(state_list)))
    print(state_list)
    return states_transitions
            

## Hopcroft minimisation

In [187]:
def transition_cross_check(state, s, partition, automata):
    '''Given a state, a generator s and a partition, this helper function checks the automaton
    to see if the state has an s-transition into a state in the partition.'''
    
    yes_s_trans_to_partition = False
    
    for st in automata:
        if st[0]==state:
            
            for trans in st[1:]:

                if trans[0]==word_nice([s]) and trans[1] in partition:
                                      
                    yes_s_trans_to_partition=True
                    break
        
            break
        
    return yes_s_trans_to_partition


def Hopcroft2(automata):
    ps = []
    pnots = []
    
    # initial split
    for state in automata:
        s_count = 0
        for trans in state[1:]:
            if trans[0]=='s':
                ps.append(state[0])
                s_count += 1
        
        if s_count==0:
            pnots.append(state[0])
    
    partition = []
    partition.append(ps)
    partition.append(pnots)
    
    num_of_splits = 1
    
    while num_of_splits > 0:
        
        num_of_splits = 0
        
        for p1 in partition:
            
            len_p1 = len(p1)
            
            # only need to check for splitting, if the partition contains more than one elementary inversion set
            if len_p1 > 1:
            
                for p2 in partition:
                    
                    for i in range(0,dim_v):

                        # checks to see if s is a transition from this partition
                        if left_length_increase(wordnice_to_vectorlist(p1[0]),s(i)):

                            ps1 = [state for state in p1 if transition_cross_check(state, s(i), p2, automata)]
                            
                            len_ps1 = len(ps1)
                            
                            # Second condition here makes sure ps1 is not p1 (the whole partition)
                            if len_ps1 > 0 and len_ps1 < len_p1:
                                p_not_s1 = [state for state in p1 if state not in ps1]

                                partition.remove(p1)
                                partition.append(ps1)
                                partition.append(p_not_s1)

                                print('Number of partitions: ')
                                print(len(partition))

                                num_of_splits += 1

                                break

                    if num_of_splits > 0:
                        break
                    else:
                        continue

                if num_of_splits > 0:
                    break
                else:
                    continue

        #print('Finished another round, Num of Splits: ')
        #print(num_of_splits)

        if num_of_splits == 0:
            break
    
    return partition
        

In [ ]:
def check_conjecture(min_automata):
    '''Check that there is a unique minimal length cone type representative'''
    for partition in min_automata:
        if len(partition) > 1:
            min_candidate = wordnice_to_vectorlist(min(partition))
            min_candidate_elementary_inversion = elementary_inversion_set(min_candidate)
            

In [201]:
elementary = elementary_roots()
elementary

[(1, 0, 0, 0),
 (0, 1, 0, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (1, 2*t1, 0, 0),
 (2*t1, 1, 0, 0),
 (0, 1, 1, 0),
 (0, 0, 1, 2*t1),
 (0, 0, 2*t1, 1),
 (1, 2*t1, 2*t1, 0),
 (2*t1, 1, 1, 0),
 (0, 1, 1, 2*t1),
 (0, 2*t1, 2*t1, 1),
 (2*t1, 2, 1, 0),
 (2*t1, 1, 1, 2*t1),
 (0, 1, 2, 2*t1),
 (2*t1, 2, 1, 2*t1),
 (2*t1, 1, 2, 2*t1)]

In [202]:
len(elementary)

18

In [203]:
autoC3tilde = bh_automata()

NUMBER OF STATES IS NOW: 13
NUMBER OF STATES IS NOW: 30
NUMBER OF STATES IS NOW: 56
NUMBER OF STATES IS NOW: 90
NUMBER OF STATES IS NOW: 128
NUMBER OF STATES IS NOW: 160
NUMBER OF STATES IS NOW: 189
NUMBER OF STATES IS NOW: 219
NUMBER OF STATES IS NOW: 242
NUMBER OF STATES IS NOW: 264
NUMBER OF STATES IS NOW: 284
NUMBER OF STATES IS NOW: 296
NUMBER OF STATES IS NOW: 306
NUMBER OF STATES IS NOW: 314
NUMBER OF STATES IS NOW: 322
NUMBER OF STATES IS NOW: 328
NUMBER OF STATES IS NOW: 332
NUMBER OF STATES IS NOW: 336
NUMBER OF STATES IS NOW: 338
NUMBER OF STATES IS NOW: 340
NUMBER OF STATES IS NOW: 342
NUMBER OF STATES IS NOW: 342
Number of states: 342
['s', 't', 'u', 'v', 'ts', 'us', 'vs', 'st', 'ut', 'vt', 'tu', 'vu', 'uv', 'sts', 'uts', 'vts', 'tus', 'vus', 'uvs', 'tst', 'ust', 'vst', 'tut', 'vut', 'uvt', 'stu', 'vtu', 'uvu', 'tuv', 'vuv', 'tsts', 'usts', 'vsts', 'tuts', 'uvts', 'stus', 'vtus', 'uvus', 'tuvs', 'vuvs', 'utst', 'vtst', 'tust', 'vust', 'uvst', 'stut', 'vtut', 'uvut', 'tuvt'

In [204]:
minimal_autoc3tilde = Hopcroft2(autoC3tilde)
minimal_autoc3tilde

Number of partitions: 
3
Number of partitions: 
4
Number of partitions: 
5
Number of partitions: 
6
Number of partitions: 
7
Number of partitions: 
8
Number of partitions: 
9
Number of partitions: 
10
Number of partitions: 
11
Number of partitions: 
12
Number of partitions: 
13
Number of partitions: 
14
Number of partitions: 
15
Number of partitions: 
16
Number of partitions: 
17
Number of partitions: 
18
Number of partitions: 
19
Number of partitions: 
20
Number of partitions: 
21
Number of partitions: 
22
Number of partitions: 
23
Number of partitions: 
24
Number of partitions: 
25
Number of partitions: 
26
Number of partitions: 
27
Number of partitions: 
28
Number of partitions: 
29
Number of partitions: 
30
Number of partitions: 
31
Number of partitions: 
32
Number of partitions: 
33
Number of partitions: 
34
Number of partitions: 
35
Number of partitions: 
36
Number of partitions: 
37
Number of partitions: 
38
Number of partitions: 
39
Number of partitions: 
40
Number of partition

Number of partitions: 
311
Number of partitions: 
312
Number of partitions: 
313
Number of partitions: 
314
Number of partitions: 
315
Number of partitions: 
316


[['ts'],
 ['s'],
 ['vu'],
 ['vsts', 'vstus', 'vstust'],
 ['stusts'],
 ['t'],
 ['uvstuvutstutsts'],
 ['uvtuvutstutsts'],
 ['uvtuvtut'],
 ['usts'],
 ['uvsts', 'uvstus'],
 ['vtuvtstuvutstutsts'],
 ['vtuvtstutsts'],
 ['vtuvtstuvtut'],
 ['vut'],
 ['uts'],
 ['tuvtstutsts'],
 ['tuvtstuvutstutsts'],
 ['tuvstuvutstutsts'],
 ['u'],
 ['uv'],
 ['vstuvutstutsts'],
 ['stuvutstutsts'],
 ['uvtuvtstutsts'],
 ['uvtuvtstuvutstutsts'],
 ['vus', 'vust'],
 ['vuvtstuvutstutsts'],
 ['vuvtstutsts'],
 ['st'],
 ['vuvtstuvutstutst'],
 ['vuvtstutst'],
 ['vuvtstuvtut'],
 ['vuvt'],
 ['v'],
 ['vtuvutstutsts'],
 ['vutstu'],
 ['vutst'],
 ['uvtus'],
 ['vtus', 'vtust'],
 ['tusts'],
 ['tuvsts'],
 ['uvtstuvutstutst'],
 ['uvtstutst'],
 ['ut'],
 ['tuvutstutsts'],
 ['tuvtus'],
 ['tuvtstuvtut'],
 ['sts'],
 ['stuvstutsts'],
 ['stuvstuvtstutsts'],
 ['stuvutstusts'],
 ['stuvstuvtstuvutstutsts'],
 ['tuvstutsts'],
 ['tuvstuvtstutsts'],
 ['tuvutstusts'],
 ['tuvstuvtstuvutstutsts'],
 ['uvstutsts'],
 ['uvstuvtstutsts'],
 ['vstutsts'],